In [2]:
from craigslist import CraigslistHousing
import pandas as pd
import numpy as np
import folium
import json
import requests 
import seaborn as sns
import plotly.figure_factory as ff
from urllib.request import urlopen
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', 50)
from matplotlib import pyplot as plt
from geopy.geocoders import Nominatim 

from geopy import distance
from geopy import Point

In [3]:
crime_df = pd.read_csv('/Users/brianmoore/Downloads/Police_Department_Incident_Reports__2018_to_Present.csv')

In [4]:
for i in crime_df:
    df_crime = crime_df.dropna(subset=['point', 'Latitude', 'Longitude'], axis=0)


df_crime['Latitude'] = df_crime['Latitude'].astype('str')
df_crime['Longitude'] = df_crime['Longitude'].astype('str')
df_crime['point'] = df_crime['point'].str.replace("[({':'})]", "")
df_crime['Incident Count'] = 1

In [5]:
crime_filt = df_crime.loc[(df_crime['Incident Year'] == 2020 )  , ['Incident Date','Incident Category', 'Analysis Neighborhood', 'point', 'Incident Count' ]]

In [6]:
crime_filt

,Incident Date,Incident Category,Analysis Neighborhood,point,Incident Count
0,2020/02/03,Missing Person,Lakeshore,"37.72694991292525, -122.47603947349434",1
1,2020/02/03,Stolen Property,Mission,"37.752439644389675, -122.41517229045435",1
2,2020/02/03,Non-Criminal,Financial District/South Beach,"37.784560141211806, -122.40733704162238",1
4,2020/01/05,Miscellaneous Investigation,Pacific Heights,"37.78711245591735, -122.44024995765258",1
5,2020/02/03,Miscellaneous Investigation,Financial District/South Beach,"37.796926429317054, -122.39950750040278",1
...,...,...,...,...,...
333874,2020/01/28,Larceny Theft,Sunset/Parkside,"37.74205567451305, -122.49818952795059",1
333887,2020/01/01,Larceny Theft,Nob Hill,"37.788222699672026, -122.42021043633935",1
333889,2020/01/10,Recovered Vehicle,Noe Valley,"37.7518282065593, -122.43855099872502",1
333892,2020/01/14,Larceny Theft,Hayes Valley,"37.77646430525107, -122.42626334607084",1


In [7]:
crime_by_neh = crime_filt.groupby(['Analysis Neighborhood']).count().sort_values(by=['Incident Count'], ascending=True)

In [8]:
del crime_by_neh['point']
del crime_by_neh['Incident Date']
del crime_by_neh['Incident Category']
crime_by_neh

,Incident Count
Analysis Neighborhood,
McLaren Park,29
Seacliff,36
Lincoln Park,62
Presidio,71
Treasure Island,92
Glen Park,134
Presidio Heights,184
Twin Peaks,187
Oceanview/Merced/Ingleside,259


In [9]:
crime_by_neh.to_csv('/Users/brianmoore/Documents/Jupyter Notebook Projects/IBM Python Tutorial notebooks/Capstone/Coursera_Capstone/sf_crime_2020.csv')

In [68]:
crime_by_neh.shape

(41, 1)

In [76]:
sf_neh_df = pd.read_csv('/Users/brianmoore/Documents/Jupyter Notebook Projects/IBM Python Tutorial notebooks/Capstone/Coursera_Capstone/Capstone_scratch/SF_neighborhood_data.csv')

In [77]:
sf_crime_comb = pd.merge(sf_neh_df, crime_by_neh, left_on='Neighborhood', right_on='Analysis Neighborhood' )

In [78]:
del sf_crime_comb['Unnamed: 0']
sf_crime_comb.sort_values(by=['Incident Count'], ascending=True)

,Neighborhood,Latitude,Longitude,Incident Count
13,McLaren Park,37.786270,-122.489248,29
23,Seacliff,37.788540,-122.486920,36
16,Lincoln Park,37.781590,-122.494410,62
34,Presidio,37.799480,-122.467270,71
36,Treasure Island,37.824890,-122.370870,92
6,Glen Park,37.738410,-122.430990,134
35,Presidio Heights,37.790280,-122.451660,184
37,Twin Peaks,37.756690,-122.442240,187
26,Oceanview/Merced/Ingleside,37.710121,-122.466961,259
24,Noe Valley,37.751600,-122.429080,297


In [70]:
piv2 = crime_filt.pivot_table(
    index=['Analysis Neighborhood'],
    columns='Incident Category', 
    values='Incident Count',
    aggfunc='count',
    margins=True)
    #dropna=False)
piv2.sort_values(by=['All'], ascending=False)

Incident Category,Arson,Assault,Burglary,Case Closure,Civil Sidewalks,Courtesy Report,Disorderly Conduct,Drug Offense,Drug Violation,Embezzlement,Family Offense,Fire Report,Forgery And Counterfeiting,Fraud,Homicide,"Human Trafficking (A), Commercial Sex Acts","Human Trafficking, Commercial Sex Acts",Juvenile Offenses,Larceny Theft,Liquor Laws,Lost Property,Malicious Mischief,Miscellaneous Investigation,Missing Person,Motor Vehicle Theft,Motor Vehicle Theft?,Non-Criminal,Offences Against The Family And Children,Other,Other Miscellaneous,Other Offenses,Prostitution,Rape,Recovered Vehicle,Robbery,Sex Offense,Stolen Property,Suicide,Suspicious,Suspicious Occ,Traffic Collision,Traffic Violation Arrest,Vandalism,Vehicle Impounded,Vehicle Misplaced,Warrant,Weapons Carrying Etc,Weapons Offence,Weapons Offense,All
Analysis Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
All,80.0,1853.0,1462.0,83.0,22.0,83.0,550.0,787.0,14.0,40.0,47.0,35.0,87.0,716.0,1.0,2.0,1.0,1.0,8815.0,10.0,748.0,1994.0,288.0,678.0,1366.0,1.0,1903.0,622.0,287.0,2410.0,237.0,44.0,7.0,800.0,870.0,67.0,175.0,16.0,9.0,576.0,67.0,392.0,56.0,26.0,26.0,1123.0,172.0,1.0,209.0,29859
Mission,11.0,257.0,137.0,1.0,3.0,12.0,85.0,75.0,9.0,7.0,5.0,3.0,6.0,50.0,NaN,1.0,1.0,1.0,822.0,3.0,92.0,205.0,38.0,57.0,166.0,NaN,197.0,63.0,29.0,412.0,25.0,21.0,1.0,75.0,120.0,6.0,19.0,NaN,1.0,53.0,13.0,96.0,7.0,4.0,5.0,183.0,50.0,NaN,35.0,3462
Tenderloin,3.0,295.0,70.0,5.0,NaN,7.0,46.0,331.0,3.0,10.0,1.0,1.0,17.0,35.0,NaN,1.0,NaN,NaN,559.0,1.0,94.0,130.0,24.0,45.0,67.0,NaN,267.0,43.0,66.0,307.0,38.0,8.0,1.0,67.0,100.0,7.0,20.0,NaN,2.0,64.0,8.0,60.0,2.0,1.0,1.0,240.0,21.0,NaN,25.0,3093
Financial District/South Beach,2.0,176.0,151.0,5.0,1.0,6.0,42.0,21.0,NaN,6.0,NaN,1.0,4.0,67.0,NaN,NaN,NaN,NaN,944.0,2.0,115.0,208.0,11.0,33.0,51.0,NaN,106.0,26.0,9.0,243.0,9.0,10.0,NaN,25.0,127.0,NaN,9.0,1.0,NaN,45.0,5.0,23.0,4.0,2.0,NaN,87.0,11.0,NaN,15.0,2603
South of Market,9.0,199.0,118.0,15.0,NaN,4.0,54.0,164.0,NaN,2.0,2.0,3.0,2.0,46.0,1.0,NaN,NaN,NaN,479.0,NaN,46.0,157.0,18.0,76.0,72.0,NaN,203.0,49.0,26.0,226.0,19.0,NaN,2.0,78.0,84.0,24.0,16.0,2.0,4.0,36.0,1.0,34.0,6.0,2.0,2.0,173.0,12.0,NaN,22.0,2488
Bayview Hunters Point,7.0,159.0,59.0,2.0,NaN,8.0,66.0,21.0,NaN,6.0,11.0,9.0,7.0,25.0,NaN,NaN,NaN,NaN,320.0,NaN,27.0,117.0,31.0,62.0,134.0,NaN,95.0,106.0,16.0,170.0,30.0,NaN,NaN,150.0,80.0,1.0,13.0,NaN,NaN,43.0,7.0,17.0,2.0,10.0,1.0,62.0,28.0,NaN,35.0,1937
Western Addition,3.0,51.0,40.0,1.0,NaN,2.0,27.0,25.0,NaN,1.0,2.0,NaN,1.0,20.0,NaN,NaN,NaN,NaN,408.0,NaN,23.0,65.0,17.0,28.0,48.0,NaN,66.0,27.0,14.0,76.0,13.0,NaN,NaN,30.0,31.0,2.0,4.0,1.0,NaN,14.0,2.0,17.0,2.0,NaN,4.0,44.0,3.0,NaN,3.0,1115
Nob Hill,4.0,65.0,55.0,1.0,1.0,NaN,15.0,9.0,NaN,NaN,NaN,1.0,3.0,30.0,NaN,NaN,NaN,NaN,297.0,NaN,34.0,67.0,3.0,24.0,39.0,NaN,47.0,21.0,9.0,92.0,4.0,4.0,NaN,14.0,34.0,1.0,10.0,NaN,NaN,10.0,1.0,10.0,2.0,NaN,1.0,28.0,3.0,NaN,2.0,941
Hayes Valley,8.0,35.0,66.0,3.0,NaN,NaN,11.0,5.0,NaN,NaN,NaN,NaN,3.0,15.0,NaN,NaN,NaN,NaN,413.0,1.0,17.0,60.0,7.0,19.0,38.0,NaN,29.0,8.0,9.0,33.0,9.0,NaN,NaN,25.0,11.0,NaN,3.0,NaN,NaN,11.0,1.0,4.0,1.0,NaN,1.0,18.0,3.0,1.0,6.0,874
